# Design attempt

## Principle

### Main idea

The system reads a bank statement file and an account book containing rules. These rules specify the expected transactions for the account, as filters on the date, object and amount of the transaction. The system tries to apply these rules to the input bank statement, asking the user to disambiguate and/or 
validate where needed. The main idea is to automatically check all the recurrent transactions, leaving
only unknown transactions for the user to manually proceed. The user can also create a new rule when
they encounter a new recurring transaction, and of course remove or disable rules which are no longer
relevant.

- The rules also allow automatic classification into a hierarchy of categories.
- The result of applying a rule is a transaction in the account book, which can be broken down into
  several operations with different categories.
- Once a bank statement is processed, the corresponding new processed transactions are added to the 
  account book.
- Eventually, a dashboard will allow more advanced analysis and visualization of existing accounts.


## File format

The account books are stored in a regular Excel .xlsx file. There can be one or multiple accounts in
a single file. The sheets are organized as follows:

- `<account>.book` contains the existing previous processed transactions, if any.
- `<account>.rules` contains the rules for this account
- `<account>.conf` contains the parameters specific to this account (e.g. bank statement format)
- (maybe) `general.conf` for common parameters
- Any other sheet with does not have an ending as `.book`, `.rules`, or `.conf` is ignored.


## Design

Generally avoiding ids and multiple tables for the sake of usability/transparency: this way the
data can be viewed and modified outside the system as an Excel sheet.

The 'book' is a dataframe in which input bank statement rows are matched with rules, resulting in
a final processed book row. Thus there are three parts for each entry:

- the input row from the bank statement, with columns bank_date, bank_text, bank_amount, bank_balance
- the rule accepted for this row, if any (columns tbd)
- the corresponding processed rows: there can be several processed rows for a single input row.
  Columns: real_date, text, amount, category (the sum of the amount must be equal to bank_amount)

Since there is potentially several processed rows for a single input bank row, and since everything is 
stored in a pandas dataframe which has to be read/written to Excel, there are empty values for cells
in the bank columns next to a non-fist processed row. These will be filled with NA.

This also means that the full dataframe is not regular, in particular with respect to sorting and
filtering. It can be transformed into two distinct dataframes: 

- one for the input bank rows (plus rule columns), which can be sorted or filtered after removing NA rows.
- another for the processed rows.


The bank statements are added first, as this is the cornerstone of the system. 

- It is not possible to add a row without any corresponding bank statement (except maybe for zero 
transactions artefacts?).
- Bank statement can be left pending, i.e. not processed yet. This allows saving the book when not
  all input rows have been processed yet.

## Rules (from 1_new_rule.py)

A rule is a dict with keys:

- N: int, interval of time expressed in time_unit
- time_unit: 'days', 'weeks', 'months', 'years'
- from_date: date, start of the rule being applied
- time_span: 'forever', 'only once', 'until ...'
- to_date: date, defined only if time_span is 'until ...'
- window: int, number of days allowed between expected date and real date
- must_happens: bool, whether the rule can or must be applied
- disabled: bool, whether the rule is currently disabled
- text_contains: str, text filter being applied
- amount_range: bool, whether the amount is a range or a single value
- amount: pair (float, float), the amount to match (absolute value)
- sign: 'any', 'credit', 'debit'

Todo: it should be connected to what to do with it: automatic or partial, 



## Tests bank statements (from accounting-tests/ipynb)

It should be possible to specify the format (even several possible formats) explicitly. Parameters:

- encoding
- separator (should be TAB if \t)
- table_first_row 
- column no for: (column nos indexed from 1)
    - col_date
    - col_object
    - col_amount (possibly two cols: cred:deb)
    - col_balance (if no column, absolute cell?)
- decimal_point_char
- date_format_pos_day
- top_most_recent


NO (The system will use its best guess for anything not specified?)

The UI could propose the detected format config and store it directly in the output file.

## Amount columns

- If single amount column, must be the amount
- If 3 amount columns, then must be cred, deb, balance. check that cred - deb = balance
- If 2 amount columns: if prev balance + amount = current balance then amount, balance; otherwise cred, deb

## About date format

The date format cannot reliably be "guessed" from a date string in all cases, even though the `dateutil` package provides a function for that (see below): in case of an ambiguous date such as 02/03/2024, the function assumes the US format (or maybe uses the locale and by default US locale?).

My approach: assume the date is made of 8 digits 2+2+4 in any order, separated by predefined possible separators (e.g. `/` or `-`). The 4 digits is obviously the year. Between the two 2 digits positions, search across rows if any of the numbers is higher than 12, marking this position as day. If no such luck, raise a warning and assume European format.
